### Change data directory below and execute cells in sequence:

In [1]:
import cv2
import imutils
import matplotlib
import numpy as np

img=cv2.imread('/scratch/user/r.kapoor/compete/ECEN649_Project_files/dataset/trainset/faces/face00001.png',
               cv2.IMREAD_GRAYSCALE)

In [2]:
detectorWidth=19
scale=1
w = detectorWidth#int(img.shape[1] / scale)
img = np.asarray(imutils.resize(img, width=w),dtype="int64")

In [3]:
#Integral Image
Matrix_S=np.reshape([np.sum(img[range(j+1),:],axis=0) for j in range(detectorWidth)],(detectorWidth,detectorWidth))
Int_image=np.transpose(np.reshape([np.sum(Matrix_S[:,range(j+1)],axis=1) 
                                   for j in range(detectorWidth)],(detectorWidth,detectorWidth)))

In [4]:
#Integral Image access function
def getIntImageValues(Int_image, index_i,index_j):
            index_valid = (index_i>=0)*(index_j>=0)
            index_i_valid=(index_valid*index_i).astype(int)
            index_j_valid=(index_valid*index_j).astype(int)
            return Int_image[index_i_valid,index_j_valid]-Int_image[0,0]*np.invert(index_valid)
A=getIntImageValues

In [5]:
def getFeature4_vertices(detectorWidth, haar_j_len,haar_k_len):
    Bottom_j=[ [j,j-np.int(haar_j_len/3)+1] for j in range(haar_j_len-1,detectorWidth) 
              for k in range(detectorWidth-haar_k_len+1)]
    Bottom_up_k=[[k,k+haar_k_len-1]
              for j in range(haar_j_len-1,detectorWidth) 
              for k in range(detectorWidth-haar_k_len+1)]
    Middle_j=[ [j-np.int(haar_j_len/3),j-np.int(2*haar_j_len/3)+1] for j in range(haar_j_len-1,detectorWidth) 
              for k in range(detectorWidth-haar_k_len+1)]
    Top_j=[ [j-np.int(2*haar_j_len/3),j-haar_j_len+1] for j in range(haar_j_len-1,detectorWidth) 
              for k in range(detectorWidth-haar_k_len+1)]
    #np.array( [j,k] for j in Up_j  for k in Up_k )
    return  (Bottom_j,Middle_j,Top_j,Bottom_up_k)

In [6]:
#Extremes of covered area
def k_min(a):
    """Min value of j for vertex (i,j) within haar feature 1"""
    return a[0]
def k_max(a):
    """Max value of j for vertex (i,j) within haar feature 1"""
    return a[1]
def j_min(a):
    """Min value of i for vertex (i,j) within haar feature 1"""
    return a[1]
def j_max(a):
    """Max value of i for vertex (i,j) within haar feature 1"""
    return a[0]

In [7]:
#detectorWidth=1
haar_J_len=np.arange(3,3*int(detectorWidth/3)+1,3)
haar_K_len=np.arange(1,detectorWidth+1)
num_features_subtypes=len([(m,n) for m in haar_J_len for n in haar_K_len])

In [8]:
def feature4_IntImage_vertices(detectorWidth, haar_j_len,haar_k_len):
    Bottom_j,Middle_j,Top_j,Bottom_up_k=getFeature4_vertices(detectorWidth, haar_j_len,haar_k_len)
    K_min=np.apply_along_axis(k_min,1,Bottom_up_k).astype(int)
    K_max=np.apply_along_axis(k_max,1,Bottom_up_k).astype(int)
    J_min=np.apply_along_axis(j_min,1,Bottom_j).astype(int)
    J_max=np.apply_along_axis(j_max,1,Bottom_j).astype(int)
    J_min_middle =np.apply_along_axis(j_min,1,Middle_j).astype(int)
    J_max_middle =np.apply_along_axis(j_max,1,Middle_j).astype(int)
    J_min_up =np.apply_along_axis(j_min,1,Top_j).astype(int)
    J_max_up =np.apply_along_axis(j_max,1,Top_j).astype(int)
    return (J_min,J_max,J_min_middle,J_max_middle,J_min_up,J_max_up,K_min,K_max)

In [9]:
my_result=[feature4_IntImage_vertices(detectorWidth, m,n)\
for m in haar_J_len for n in haar_K_len]

In [10]:
J_min=np.array([]).astype(int)
J_max=np.array([]).astype(int)
J_min_middle =np.array([]).astype(int)
J_max_middle =np.array([]).astype(int)
J_min_up =np.array([]).astype(int)
J_max_up =np.array([]).astype(int)
K_min=np.array([]).astype(int)
K_max=np.array([]).astype(int)
for j in range(num_features_subtypes):
    J_min = np.append(J_min,my_result[j][0]).astype(int)
    J_max = np.append(J_max,my_result[j][1]).astype(int)
    J_min_middle = np.append(J_min_middle,my_result[j][2]).astype(int)
    J_max_middle = np.append(J_max_middle,my_result[j][3]).astype(int)
    J_min_up = np.append(J_min_up,my_result[j][4]).astype(int)
    J_max_up = np.append(J_max_up,my_result[j][5]).astype(int)
    K_min = np.append(K_min,my_result[j][6]).astype(int)
    K_max = np.append(K_max,my_result[j][7]).astype(int)

In [11]:
import pandas as pd
Feature4_indices = pd.DataFrame({ 'J_min':J_min, 'J_max':J_max,\
                                'J_min_middle':J_min_middle, 'J_max_middle':J_max_middle,\
                                'J_min_up':J_min_up, 'J_max_up':J_max_up,\
                                'K_min': K_min, 'K_max': K_max,})
Feature4_indices.to_csv("Int_image_indices/Feature4_indices.csv")

In [12]:
def getFeature4Values(Int_image, J_min,J_max,J_min_middle,J_max_middle,J_min_up,J_max_up,K_min,K_max):
            Top_area=A(Int_image,J_max_up,K_max)-A(Int_image,J_max_up,K_min-1)-\
A(Int_image,J_min_up-1,K_max)+A(Int_image,J_min_up-1,K_min-1)
            Middle_area=A(Int_image,J_max_middle,K_max)-A(Int_image,J_max_middle,K_min-1)-\
A(Int_image,J_min_middle-1,K_max)+A(Int_image,J_min_middle-1,K_min-1)
            Bottom_area=A(Int_image,J_max,K_max)-A(Int_image,J_max,K_min-1)-\
A(Int_image,J_min-1,K_max)+A(Int_image,J_min-1,K_min-1)
            return Top_area+Bottom_area-Middle_area

In [13]:
Feature4=getFeature4Values(Int_image, J_min,J_max,J_min_middle,J_max_middle,J_min_up,J_max_up,K_min,K_max)

In [14]:
Feature4.shape

(10830,)